In [ ]:
import kwant
import semicon

import numpy as np
import scipy.linalg as la
import scipy.sparse
import scipy.sparse.linalg as sla
import pickle

import sympy
sympy.init_printing(print_builtin=False)

import matplotlib.pyplot as plt
%matplotlib inline

import codes.higher_order_lowdin as lowdin
from codes.combine import apply_peierls_to_template, magnetic_perturbation

In [ ]:
# %load_ext line_profiler

# k.p example of using the Lowdin perturbation

This example requires [semicon](https://gitlab.kwant-project.org/semicon/semicon) to be installed.

It should be as easy as 
```
pip install git+https://gitlab.kwant-project.org/semicon/semicon.git
```

In [ ]:
try:
    import semicon
except ImportError:
    print("Semicon should be installed to run this notebook.")

In [ ]:
import kwant
import scipy
import sympy
import semicon


print("semicon version:", semicon.__version__)
print("kwant version:", kwant.__version__)
print("scipy version:", scipy.__version__)
print("sympy version", sympy.__version__)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=kwant.KwantDeprecationWarning)

## Prepare k.p model and solve it exactly

In [ ]:
gamma_0 = 1.0

# Get the k.p model for ZincBlende material
model = semicon.models.ZincBlende(
    components=('foreman',),
    default_databank='lawaetz'
)
kpham = model.hamiltonian

# Get parameters for InAs
InAs = model.parameters('InAs').renormalize(new_gamma_0=gamma_0)

display(kpham)

# Add Zeeman field with InAs bulk g-factor value
g_bulk = -15

kpham += sum(-sympy.sympify(B) * g_bulk * sympy.sympify('mu_B') * model.spin_operators[i] for i, B in enumerate(['B_x', 'B_y', 'B_z']))

# Add electric field
kpham += sympy.sympify('- E_x * x - E_y * y') * np.eye(8)

InAs = {**InAs, 'B_x': 0, 'B_y': 0, 'B_z': 0, 'E_x': 0, 'E_y': 0}

In [ ]:
kpham

In [ ]:
# Discretize the model to get a kwant tight-binding model
grid_spacing = 0.2
R = 5

tbham = kwant.continuum.discretize(kpham, grid=grid_spacing)
# Add orbital magnetic field
tbham = apply_peierls_to_template(tbham)

# Fill an infinite wire with circular cross section with the model
shape = lambda site: la.norm(site.pos) < R
tz = np.array(tbham.symmetry.periods)[-1]
wire = kwant.Builder(symmetry=kwant.TranslationalSymmetry(tz))
wire.fill(tbham, shape, start=(0, 0, 0))
kwant.plotter.plot(wire, num_lead_cells=10, pos_transform=lambda pos: pos[:2]);

# Get the unit cell and hopping matrices
wiref = wire.finalized()
Hos = wiref.cell_hamiltonian(params=InAs, sparse=True).tocsr()
Hos.eliminate_zeros()
Hhop = wiref.inter_cell_hopping(params=InAs, sparse=True).tocsr()
Hhop.eliminate_zeros()

# Make second order model in k_z
H0 = lowdin.Model({1: Hos + (Hhop + Hhop.T.conj())})
Hkz = lowdin.Model({'k_z': grid_spacing * (1j * Hhop - 1j * Hhop.T.conj()),
                    'k_z**2': grid_spacing**2 * (-1) * (Hhop + Hhop.T.conj())})
H = H0 + Hkz

In [ ]:
def exact_spectrum(wire, ks, params, sigma=0.5, num_states=6):
    wrapped_wire = kwant.wraparound.wraparound(wire).finalized()
    es = []
    a = params['grid_spacing']
    for k in ks:
        pars = {**params, 'k_x': k * a}
        e = scipy.sparse.linalg.eigsh(wrapped_wire.hamiltonian_submatrix(params=pars, sparse=True),
                                      return_eigenvectors=False, sigma=sigma, k=num_states)
        e = sorted(e)
        es.append(e)
    return np.array(es)

In [ ]:
%%time
# exact eigenenergies
ks = np.linspace(-0.1, 0.1, 21)

E_x = 0
B_z = 0
# This takes long, precalculate it
try:
    es = pickle.load(open('wire_exact_0.pickle', 'rb'))['es']
except:
    params = InAs.copy()
    params.update(E_x=E_x, B_z=B_z, grid_spacing=grid_spacing)
    es = exact_spectrum(wire, ks, params, sigma=0.5, num_states=6)
    pickle.dump(dict(es=es,
                     params=params,
                     R=R,
                     grid_spacing=grid_spacing),
        open('wire_exact_0.pickle', 'wb'))
    
plt.plot(ks, es, '-');

## Build perturbative Hamiltonian

In [ ]:
def electric_terms(wiref):
    # Make electric field perturbations

    xop = kwant.operator.Density(wiref, onsite=lambda site: site.pos[0] * np.eye(8), sum=True).tocoo().tocsr()
    xop.eliminate_zeros()
    yop = kwant.operator.Density(wiref, onsite=lambda site: site.pos[1] * np.eye(8), sum=True).tocoo().tocsr()
    yop.eliminate_zeros()

    V = lowdin.Model({'E_x': -xop, 'E_y': -yop})
    
    return V

def magnetic_terms(tbham, shape, order=1):
    # Make magnetic field perturbations
    HB = 0
    for i, (n, B) in enumerate(zip(np.eye(3), ['B_x', 'B_y', 'B_z'])):
        # orbital magnetic field
        for k in range(1, order + 1):
            tbhamB = magnetic_perturbation(tbham, n=n, order=k)
            wireB = kwant.Builder(symmetry=kwant.TranslationalSymmetry(tz))
            wireB.fill(tbhamB, shape, start=(0, 0, 0))
            wireB = wireB.finalized()
            matB = wireB.cell_hamiltonian(params=InAs, sparse=True).tocsr()
            matB += wireB.inter_cell_hopping(params=InAs, sparse=True).tocsr() + wireB.inter_cell_hopping(params=InAs, sparse=True).tocsr().T.conj()
            HB += lowdin.Model({B + '**{}'.format(k): matB})
        # Zeeman term
        matZ = kwant.operator.Density(wiref, onsite=lambda site: model.spin_operators[i], sum=True).tocoo().tocsr()
        matZ.eliminate_zeros()
        HB += lowdin.Model({B: InAs['mu_B'] * g_bulk * matZ})

    return HB

In [ ]:
sz0 = 0.5 * np.diag([1, -1, 3, 1, -1, -3, -1, 1])

sz = kwant.operator.Density(wiref, 
                            onsite=lambda site: sz0,
                            sum=True).tocoo().tocsr()

def fix_basis(E0, vecsA):
    # Order eigenstates by energy
    e_order = np.argsort(E0)
    E02 = np.sort(E0)
    vecsA2 = vecsA[:, e_order]
    # Make sure that the degenerate states are orthogonal
    vecsA2, _ = la.qr(vecsA2, mode='economic')
    assert len(E02) % 2 == 0
    # Fix basis in degenerate eigensubspace by spin z polarization
    szA = vecsA2.T.conj() @ sz @ vecsA2
    for i in range(0, len(E02), 2):
        assert np.isclose(E02[i], E02[i+1]), (i, E02)
        _, U = la.eigh(szA[i:i+2, i:i+2])
        vecsA2[:, i:i+2] = vecsA2[:, i:i+2] @ U
        szAi = vecsA2[:, i:i+2].T.conj() @ sz @ vecsA2[:, i:i+2]
        assert np.allclose(szAi, np.diag(np.diag(szAi)))
    # fix phases, this is arbitrary, but removes phase ambiguity
    phases = np.diag(np.angle(vecsA2.T @ vecsA2))
    vecsA2 = vecsA2 @ np.diag(np.exp(-1j*phases/2))
    return E02, vecsA2

### Lowest subband $l=0$ states

In [ ]:
# E_x = 0.01 # V/nm
# B_z = 0.25 # Tesla
E_x = 0.015 # V/nm
B_z = 0.25 # Tesla

#### 2nd order in most parameters, linear in B

In [ ]:
%%time
# Make perturbation basis of two lowest states
kpm_params = dict(num_moments=1000)

E0, vecsA = scipy.sparse.linalg.eigsh(H0[1], return_eigenvectors=True, sigma=0.45, k=2)
E0, vecsA = fix_basis(E0, vecsA)
print(E0)

V = electric_terms(wiref)
HB = magnetic_terms(tbham, shape)

# only keep most interesting keys
intersting_keys = sympy.sympify(['k_z', 'k_z**2',
                                 'E_x', 'E_y', 'E_x**2', 'E_y**2',
                                 'B_x', 'B_y', 'B_z',
                                 'E_x * k_z', 'E_y * k_z',
                                ])

model0 = lowdin.effective_model(H0, Hkz + V + HB, evec_A=vecsA, order=2,
                                kpm_params=kpm_params,
                                interesting_keys=intersting_keys)

display(model0.around(decimals=4).tosympy())

In [ ]:
%%time
# effective eigenenergies with electric and magnetic fields
es0 = []

ks = np.linspace(-0.1, 0.1, 101)
for k in ks:
    pars = {'k_z': k, 'E_x': E_x, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': B_z}
    e = la.eigh((model0).subs(pars)[1], eigvals_only=True)
    es0.append(e)
es2 = np.array(es0)

# This takes long, precalculate it
filename = 'wire_exact_{}_{}.pickle'.format(E_x, B_z)
try:
    esEB = pickle.load(open(filename, 'rb'))['esEB']
except:
    params = InAs.copy()
    params.update(E_x=E_x, B_z=B_z, grid_spacing=grid_spacing)
    esEB = exact_spectrum(wire, ks, params, sigma=0.42, num_states=2)
    pickle.dump(dict(esEB=esEB,
                     params=params,
                     R=R,
                     grid_spacing=grid_spacing),
        open(filename, 'wb'))
    
plt.plot(ks, esEB, '--')
plt.plot(ks, es2, '-')

In [ ]:
plt.plot(ks, esEB, '--')
plt.plot(ks, es2, '-')
plt.ylim([0.545, 0.547])
plt.xlim([-0.04, 0.04])

#### 2nd order in all parameters

In [ ]:
%%time
# Make perturbation basis of two lowest states
kpm_params = dict(num_moments=1000)

E0, vecsA = scipy.sparse.linalg.eigsh(H0[1], return_eigenvectors=True, sigma=0.45, k=2)
E0, vecsA = fix_basis(E0, vecsA)
print(E0)

V = electric_terms(wiref)
HB = magnetic_terms(tbham, shape, order=2)

# only keep up to correct power
intersting_keys = lowdin._interesting_keys(sympy.sympify(['k_z', 'E_x', 'E_y', 'B_x', 'B_y', 'B_z']), order=2)

model0 = lowdin.effective_model(H0, Hkz + V + HB, evec_A=vecsA, order=2,
                                kpm_params=kpm_params,
                                interesting_keys=intersting_keys)

display(model0.around(decimals=4).tosympy())

In [ ]:
%%time
# effective eigenenergies with electric and magnetic fields
es0 = []

ks = np.linspace(-0.1, 0.1, 101)
for k in ks:
    pars = {'k_z': k, 'E_x': E_x, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': B_z}
    e = la.eigh((model0).subs(pars)[1], eigvals_only=True)
    es0.append(e)
es2 = np.array(es0)

# This takes long, precalculate it
filename = 'wire_exact_{}_{}.pickle'.format(E_x, B_z)
try:
    esEB = pickle.load(open(filename, 'rb'))['esEB']
except:
    params = InAs.copy()
    params.update(E_x=E_x, B_z=B_z, grid_spacing=grid_spacing)
    esEB = exact_spectrum(wire, ks, params, sigma=0.42, num_states=2)
    pickle.dump(dict(esEB=esEB,
                     params=params,
                     R=R,
                     grid_spacing=grid_spacing),
        open(filename, 'wb'))
    
plt.plot(ks, esEB, '--')
plt.plot(ks, es2, '-')

In [ ]:
plt.plot(ks, esEB, '--')
plt.plot(ks, es2, '-')
plt.ylim([0.545, 0.547])
plt.xlim([-0.04, 0.04])

#### 4th order in $E$, 2nd order in everything

In [ ]:
%%time
# Make perturbation basis of two lowest states
kpm_params = dict(num_moments=1000)

E0, vecsA = scipy.sparse.linalg.eigsh(H0[1], return_eigenvectors=True, sigma=0.45, k=2)
E0, vecsA = fix_basis(E0, vecsA)
print(E0)

V = electric_terms(wiref)
HB = magnetic_terms(tbham, shape)

# only keep most interesting keys
intersting_keys = (list(lowdin._interesting_keys(sympy.sympify(['k_z', 'E_x', 'E_y', 'B_x', 'B_y', 'B_z']), order=2))
                  + sympy.sympify(['E_x**3', 'E_x**4']))

model0 = lowdin.effective_model(H0, Hkz + V + HB, evec_A=vecsA, order=4,
                                kpm_params=kpm_params,
                                interesting_keys=intersting_keys)

display(model0.around(decimals=6).tosympy())

In [ ]:
%%time
# effective eigenenergies with electric and magnetic fields
es0 = []
ks = np.linspace(-0.1, 0.1, 101)
for k in ks:
    pars = {'k_z': k, 'E_x': E_x, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': B_z}
    e = la.eigh((model0).subs(pars)[1], eigvals_only=True)
    es0.append(e)
es42 = np.array(es0)

# This takes long, precalculate it
filename = 'wire_exact_{}_{}.pickle'.format(E_x, B_z)
try:
    esEB = pickle.load(open(filename, 'rb'))['esEB']
except:
    params = InAs.copy()
    params.update(E_x=E_x, B_z=B_z, grid_spacing=grid_spacing)
    esEB = exact_spectrum(wire, ks, params, sigma=0.42, num_states=2)
    pickle.dump(dict(esEB=esEB,
                     params=params,
                     R=R,
                     grid_spacing=grid_spacing),
        open(filename, 'wb'))
    
plt.plot(ks, esEB, '--')
plt.plot(ks, es42, '-')

In [ ]:
plt.plot(ks, esEB, '--')
plt.plot(ks, es4, '-')
plt.ylim([0.545, 0.547])
plt.xlim([-0.04, 0.04])

#### 4th order in everything

In [ ]:
%%time
# Make perturbation basis of two lowest states
kpm_params = dict(num_moments=1000)

E0, vecsA = scipy.sparse.linalg.eigsh(H0[1], return_eigenvectors=True, sigma=0.45, k=2)
E0, vecsA = fix_basis(E0, vecsA)
print(E0)

V = electric_terms(wiref)
HB = magnetic_terms(tbham, shape)

# only keep most interesting keys
intersting_keys = lowdin._interesting_keys(sympy.sympify(['k_z', 'E_x', 'E_y', 'B_x', 'B_y', 'B_z']), order=4)

model0 = lowdin.effective_model(H0, Hkz + V + HB, evec_A=vecsA, order=4,
                                kpm_params=kpm_params,
                                interesting_keys=intersting_keys)

display(model0.around(decimals=6).tosympy())

In [ ]:
%%time
# effective eigenenergies with electric and magnetic fields
es0 = []
ks = np.linspace(-0.1, 0.1, 101)
for k in ks:
    pars = {'k_z': k, 'E_x': E_x, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': B_z}
    e = la.eigh((model0).subs(pars)[1], eigvals_only=True)
    es0.append(e)
es4 = np.array(es0)

# This takes long, precalculate it
filename = 'wire_exact_{}_{}.pickle'.format(E_x, B_z)
try:
    esEB = pickle.load(open(filename, 'rb'))['esEB']
except:
    params = InAs.copy()
    params.update(E_x=E_x, B_z=B_z, grid_spacing=grid_spacing)
    esEB = exact_spectrum(wire, ks, params, sigma=0.42, num_states=2)
    pickle.dump(dict(esEB=esEB,
                     params=params,
                     R=R,
                     grid_spacing=grid_spacing),
        open(filename, 'wb'))
    
plt.plot(ks, esEB, '--')
plt.plot(ks, es4, '-')

In [ ]:
plt.plot(ks, esEB, '--')
plt.plot(ks, es4, '-')
plt.ylim([0.545, 0.547])
plt.xlim([-0.04, 0.04])

#### 6th order in $E$, 2nd order in everything

In [ ]:
%%time
# Make perturbation basis of two lowest states
kpm_params = dict(num_moments=1000)

E0, vecsA = scipy.sparse.linalg.eigsh(H0[1], return_eigenvectors=True, sigma=0.45, k=2)
E0, vecsA = fix_basis(E0, vecsA)
print(E0)

V = electric_terms(wiref)
HB = magnetic_terms(tbham, shape)

# only keep most interesting keys
intersting_keys = sympy.sympify(['k_z', 'k_z**2',
                                 'E_x', 'E_y', 'E_x**2', 'E_y**2',
                                 'E_x**3', 'E_x**4', 'E_x**5', 'E_x**6',
                                 'E_x**2 * k_z', 'E_x * k_z**2', 'E_x**2 * k_z**2',
                                 'E_x**3 * k_z', 'E_x * k_z**3',
                                 'B_x', 'B_y', 'B_z',
                                 'E_x * k_z', 'E_y * k_z',
                                ])

model0 = lowdin.effective_model(H0, Hkz + V + HB, evec_A=vecsA, order=6,
                                kpm_params=kpm_params,
                                interesting_keys=intersting_keys)

display(model0.around(decimals=4).tosympy())

In [ ]:
%%time
# effective eigenenergies with electric and magnetic fields
es0 = []
ks = np.linspace(-0.1, 0.1, 101)
for k in ks:
    pars = {'k_z': k, 'E_x': E_x, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': B_z}
    e = la.eigh((model0).subs(pars)[1], eigvals_only=True)
    es0.append(e)
es6 = np.array(es0)

# This takes long, precalculate it
filename = 'wire_exact_{}_{}.pickle'.format(E_x, B_z, grid_spacing=grid_spacing)
try:
    esEB = pickle.load(open(filename, 'rb'))['esEB']
except:
    params = InAs.copy()
    params.update(E_x=E_x, B_z=B_z)
    esEB = exact_spectrum(wire, ks, params, sigma=0.42, num_states=2)
    pickle.dump(dict(esEB=esEB,
                     params=params,
                     R=R,
                     grid_spacing=grid_spacing),
        open(filename, 'wb'))
    
plt.plot(ks, esEB, '--')
plt.plot(ks, es6, '-')

In [ ]:
plt.plot(ks, esEB, '--')
plt.plot(ks, es6, '-')
plt.ylim([0.545, 0.547])
plt.xlim([-0.04, 0.04])

#### 6th order in $E$, $E^2$ correction to effective mass

In [ ]:
%%time
# Make perturbation basis of two lowest states
kpm_params = dict(num_moments=1000)

E0, vecsA = scipy.sparse.linalg.eigsh(H0[1], return_eigenvectors=True, sigma=0.45, k=2)
E0, vecsA = fix_basis(E0, vecsA)
print(E0)

V = electric_terms(wiref)
HB = magnetic_terms(tbham, shape)

# only keep most interesting keys
intersting_keys = sympy.sympify(['k_z', 'k_z**2',
                                 'E_x', 'E_y', 'E_x**2', 'E_y**2',
                                 'E_x**3', 'E_x**4', 'E_x**5', 'E_x**6',
                                 'E_x**2 * k_z', 'E_x**2 * k_z**2',
                                 'B_x', 'B_y', 'B_z',
                                 'E_x * k_z', 'E_y * k_z',
                                ])

model0 = lowdin.effective_model(H0, Hkz + V + HB, evec_A=vecsA, order=6,
                                kpm_params=kpm_params,
                                interesting_keys=intersting_keys)

model0.around(decimals=4).tosympy()

In [ ]:
%%time
# effective eigenenergies with electric and magnetic fields
es0 = []
ks = np.linspace(-0.1, 0.1, 101)
for k in ks:
    pars = {'k_z': k, 'E_x': E_x, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': B_z}
    e = la.eigh((model0).subs(pars)[1], eigvals_only=True)
    es0.append(e)
es62 = np.array(es0)

# This takes long, precalculate it
filename = 'wire_exact_{}_{}.pickle'.format(E_x, B_z)
try:
    esEB = pickle.load(open(filename, 'rb'))['esEB']
except:
    params = InAs.copy()
    params.update(E_x=E_x, B_z=B_z, grid_spacing=grid_spacing)
    esEB = exact_spectrum(wire, ks, params, sigma=0.42, num_states=2)
    pickle.dump(dict(esEB=esEB,
                     params=params,
                     R=R,
                     grid_spacing=grid_spacing),
        open(filename, 'wb'))
    
plt.plot(ks, esEB, '--')
plt.plot(ks, es62, '-')

In [ ]:
plt.plot(ks, esEB, '--')
plt.plot(ks, es62, '-')
plt.ylim([0.545, 0.547])
plt.xlim([-0.04, 0.04])

In [ ]:
for i in range(2):
    plt.plot(ks, 1000*esEB[:, i], '-', c='C0', label=('exact' if i == 0 else ''))
    plt.plot(ks, 1000*es2[:, i], ':', c='C1', label=('2nd order' if i == 0 else ''))
    plt.plot(ks, 1000*es42[:, i], '--', c='C2', label=(r'2nd order + $E_x^4$' if i == 0 else ''))

plt.ylim([545, 547])
plt.xlim([-0.05, 0.05])
plt.legend()
plt.ylabel(r'$E$ [meV]')
plt.xlabel(r'$k_z$ [nm$^{-1}$]')
plt.tight_layout()
plt.savefig('nanowire_spectrum.pdf')

In [ ]:
plt.plot(ks, esEB, '--')
plt.plot(ks, es2, ':')
plt.plot(ks, es4, ':')
plt.plot(ks, es6, '-')
plt.plot(ks, es62, '-')

plt.ylim([0.545, 0.548])
plt.xlim([-0.05, 0.05])
plt.legend(['exact',
            'exact',
            '2nd order',
            '2nd order',
            r'2nd order + $E^4$',
            r'2nd order + $E^4$',
            r'2nd order + $E^6$',
            r'2nd order + $E^6$',
            r'2nd order + $E^6$ + $E^6 k_z^2$',
            r'2nd order + $E^6$ + $E^6 k_z^2$'],
           loc=(1, 0))

In [ ]:
# Use more states exactly
kpm_params = dict(num_moments=100)

# Make perturbation basis of two lowest states
E0, vecs = scipy.sparse.linalg.eigsh(H0[1], return_eigenvectors=True, sigma=0.4, k=60)
E0, vecs = fix_basis(E0, vecs)
print(E0)

ind_0 = np.argwhere(E0 > 0)[:2].flatten()

model0 = lowdin.effective_model(H0, Hkz + V + HB,
                                evec_A=vecs[:, ind_0],
                                evec_B=vecs[:, np.setdiff1d(np.arange(vecs.shape[1]), ind_0)],
                                order=2, interesting_keys=intersting_keys,
                                kpm_params=kpm_params)
display(model0.around(decimals=4).tosympy())

In [ ]:
# effective eigenenergies with electric and magnetic fields
es0 = []
E_x = 0.02 # V/nm
B_z = 0.5 # Tesla
ks = np.linspace(-0.1, 0.1, 101)
for k in ks:
    pars = {'k_z': k, 'E_x': E_x, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': B_z}
    e = la.eigh((model0).subs(pars)[1], eigvals_only=True)
    es0.append(e)
es0 = np.array(es0)
    
plt.plot(ks, esEB, '--')
plt.plot(ks, es0, '-')

In [ ]:
# Make perturbation basis of two lowest states
kpm_params = dict(num_moments=1000)

E0, vecsA = scipy.sparse.linalg.eigsh(H0[1], return_eigenvectors=True, sigma=0.45, k=2)
E0, vecsA = fix_basis(E0, vecsA)
print(E0)

V = electric_terms(wiref)
HB = magnetic_terms(tbham, shape)

# only keep most interesting keys
intersting_keys = sympy.sympify(['k_z', 'k_z**2',
                                 'E_x', 'E_y', 'E_x**2', 'E_y**2',
                                 'E_x**3', 'E_x**4', 'E_x**5', 'E_x**6',
                                 'B_x', 'B_y', 'B_z',
                                 'E_x * k_z', 'E_y * k_z'])

model0 = lowdin.effective_model(H0, Hkz + V + HB, evec_A=vecsA, order=6,
                                kpm_params=kpm_params,
                                interesting_keys=intersting_keys)

model0.around(decimals=4).tosympy()

In [ ]:
es00 = []
# es00e = []
Exs = np.linspace(-0.02, 0.02, 41)
for E_x in Exs:
    print(E_x)
    pars = {'k_z': 0, 'E_x': E_x, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': 0}
    e = la.eigh((model0).subs(pars)[1], eigvals_only=True)
    es00.append(e)

#     params = InAs.copy()
#     params.update(E_x=E_x)
#     ee = exact_spectrum(wire, [0], params, sigma=0.42, num_states=2)
#     es00e.append(ee)
es00 = np.array(es00)
# es00e = np.array(es00e)

In [ ]:
# with 4th order
plt.plot(Exs, es002[:, 0], '-')
plt.plot(Exs, es004[:, 0], '-')
plt.plot(Exs, es006[:, 0], '-')
plt.plot(Exs, es00e.reshape((41, 2))[:, 0], '--')
plt.ylim([0.4, 0.47])
plt.xlim([-0.02, 0.02])
plt.legend(['2nd order', '4th order', '6th order', 'exact'])
plt.xlabel(r'$E_x$')
plt.ylabel(r'$E$')

In [ ]:
# Shift exact spectrum to match it
plt.plot(ks, esEB - 0.0248, '--')
plt.plot(ks, es0, '-')
plt.ylim([0.375, 0.39])
plt.xlim([-0.05, 0.05])

### Second lowest subband $l=1$ states

In [ ]:
# Make perturbation basis of second lowest states
kpm_params = dict(num_moments=1000)
E0, vecsA = scipy.sparse.linalg.eigsh(H0[1], return_eigenvectors=True, sigma=0.55, k=4)
E0, vecsA = fix_basis(E0, vecsA)
print(E0)
print(np.diag(vecsA.T.conj() @ sz @ vecsA))
print(np.diag(vecsA.T.conj() @ Hos @ vecsA))

model1 = lowdin.effective_model(H0, Hkz + V + HB, evec_A=vecsA,
                                order=2, interesting_keys=intersting_keys,
                                kpm_params=kpm_params)

model1.around(decimals=4).tosympy()

In [ ]:
# effective eigenenergies with zero fields
es1 = []
ks = np.linspace(-0.1, 0.1, 101)
for k in ks:
    pars = {'k_z': k, 'E_x': 0, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': 0}
    e = la.eigh((model1).subs(pars)[1], eigvals_only=True)
    es1.append(e)
es1 = np.array(es1)
    
plt.plot(ks, es1, '-')

In [ ]:
# effective eigenenergies with electric field
E_x = 0.01
es1 = []
ks = np.linspace(-0.1, 0.1, 101)
for k in ks:
    pars = {'k_z': k, 'E_x': E_x, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': 0}
    e = la.eigh((model1).subs(pars)[1], eigvals_only=True)
    es1.append(e)
es1 = np.array(es1)

plt.plot(ks, es1, '-')

In [ ]:
# effective eigenenergies with electric and magnetic fields
es1 = []
E_x = 0.02
B_z = 0.5
ks = np.linspace(-0.1, 0.1, 101)
for k in ks:
    pars = {'k_z': k, 'E_x': E_x, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': B_z}
    e = la.eigh((model1).subs(pars)[1], eigvals_only=True)
    es1.append(e)
es1 = np.array(es1)

plt.plot(ks, es1, '-')
plt.ylim([0.48, 0.50])

## Perturbation around finite $E_x$

### Lowest subband $l=0$ states

In [ ]:
# E_x = 0.01 # V/nm
# B_z = 0.25 # Tesla
E_x0 = 0.1 # V/nm
dE_x = 0 # V/nm
B_z = 1 # Tesla

#### 2nd order in most parameters, linear in B

In [ ]:
H0[1].shape, xop.shape

In [ ]:
%%time
# Make perturbation basis of two lowest states
kpm_params = dict(num_moments=1000)

xop = kwant.operator.Density(wiref, onsite=lambda site: site.pos[0] * np.eye(8), sum=True).tocoo().tocsr()
xop.eliminate_zeros()

E0, vecsA = scipy.sparse.linalg.eigsh(H0[1] - E_x0 * xop, return_eigenvectors=True, sigma=0.45, k=2)
E0, vecsA = fix_basis(E0, vecsA)
print(E0)

V = electric_terms(wiref)
HB = magnetic_terms(tbham, shape)

# only keep most interesting keys
intersting_keys = sympy.sympify(['k_z', 'k_z**2',
                                 'E_x', 'E_y', 'E_x**2', 'E_y**2',
                                 'B_x', 'B_y', 'B_z',
                                 'E_x * k_z', 'E_y * k_z',
                                ])

model2f = lowdin.effective_model(H0 - E_x0 * xop, Hkz + V + HB, evec_A=vecsA, order=2,
                                kpm_params=kpm_params,
                                interesting_keys=intersting_keys)

display(model2f.around(decimals=4).tosympy())

In [ ]:
%%time
# effective eigenenergies with electric and magnetic fields
es0 = []

ks = np.linspace(-0.1, 0.1, 101)
for k in ks:
    pars = {'k_z': k, 'E_x': dE_x, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': B_z}
    e = la.eigh((model2f).subs(pars)[1], eigvals_only=True)
    es0.append(e)
es2f = np.array(es0)

es0 = []
for k in ks:
    pars = {'k_z': k, 'E_x': dE_x, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': 0}
    e = la.eigh((model2f).subs(pars)[1], eigvals_only=True)
    es0.append(e)
es20 = np.array(es0)

# This takes long, precalculate it
filename = 'wire_exact_finite_E_{}_{}.pickle'.format(E_x0, B_z)
try:
    esEB = pickle.load(open(filename, 'rb'))['esEB']
except:
    params = InAs.copy()
    params.update(E_x=E_x0, B_z=B_z, grid_spacing=grid_spacing)
    esEB = exact_spectrum(wire, ks, params, sigma=0.40, num_states=2)
    pickle.dump(dict(esEB=esEB,
                     params=params,
                     R=R,
                     grid_spacing=grid_spacing),
        open(filename, 'wb'))
    
plt.plot(ks, esEB, '--')
plt.plot(ks, es2f, '-')
plt.plot(ks, es20, ':')

In [ ]:
plt.rcParams['text.usetex'] = True

for i in range(2):
    plt.plot(ks, 1000*es2f[:, i], '-', c='C2', label=(r'2nd order at $B_z = 1$T' if i == 0 else ''))
    plt.plot(ks, 1000*es20[:, i], ':', c='C3', label=('2nd order at $B_z = 0$' if i == 0 else ''))
    plt.plot(ks, 1000*esEB[:, i], '--', c='C1', label=('exact at $B_z = 1$T' if i == 0 else ''))
    # plt.plot(ks, 1000*es2[:, i] + 052.3, '-.', c='C4', label=(r'2nd order at $B_z = 1$T expanded around $E_x = 0$' if i == 0 else ''))

plt.ylim([399, 410])
plt.xlim([-0.1, 0.1])
plt.legend()
plt.ylabel(r'$E$ [meV]')
plt.xlabel(r'$k_z$ [nm$^{-1}$]')
plt.tight_layout()
plt.savefig('nanowire_spectrum.pdf')

#### Compare with 2nd order in most parameters, linear in B expanded around $E_x=0$

In [ ]:
%%time
# Make perturbation basis of two lowest states
kpm_params = dict(num_moments=1000)

E0, vecsA = scipy.sparse.linalg.eigsh(H0[1], return_eigenvectors=True, sigma=0.45, k=2)
E0, vecsA = fix_basis(E0, vecsA)
print(E0)

V = electric_terms(wiref)
HB = magnetic_terms(tbham, shape)

# only keep most interesting keys
intersting_keys = sympy.sympify(['k_z', 'k_z**2',
                                 'E_x', 'E_y', 'E_x**2', 'E_y**2',
                                 'B_x', 'B_y', 'B_z',
                                 'E_x * k_z', 'E_y * k_z',
                                ])

model0 = lowdin.effective_model(H0, Hkz + V + HB, evec_A=vecsA, order=2,
                                kpm_params=kpm_params,
                                interesting_keys=intersting_keys)

display(model0.around(decimals=5).tosympy())

In [ ]:
# get parameters in convenient units
# effective mass in free electron mass units
print('m^* =', params['hbar']**2 / (2 * model0['k_z**2'][0, 0].real * params['m_0']), r'm_0')

# g-factor
print('g_xx = g_yy =', -2 * model0['B_x'][1, 0].real / params['mu_B'])
print('g_zz =', -2 * model0['B_z'][0, 0].real / params['mu_B'])

In [ ]:
%%time
# effective eigenenergies with electric and magnetic fields
es0 = []

ks = np.linspace(-0.1, 0.1, 101)
for k in ks:
    pars = {'k_z': k, 'E_x': E_x0, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': B_z}
    e = la.eigh((model0).subs(pars)[1], eigvals_only=True)
    es0.append(e)
es2 = np.array(es0)

plt.plot(ks, esEB, '--')
plt.plot(ks, es2 + 0.0522, '-')

In [ ]:
xop = kwant.operator.Density(wiref, onsite=lambda site: site.pos[0] * np.eye(8), sum=True).tocoo().tocsr()
xop.eliminate_zeros()

In [ ]:
kphamx = sympy.Matrix(sympy.sympify('x') * np.eye(8))
tbhamx = kwant.continuum.discretize(kphamx, coords=['x', 'y', 'z'], grid_spacing=grid_spacing)
tbhamx[list(tbhamx.sites())[0].family.neighbors()] = np.zeros((8, 8))

# Fill an infinite wire with circular cross section with the model
shape = lambda site: la.norm(site.pos) < R
tz = np.array(tbham.symmetry.periods)[-1]
wirex = kwant.Builder(symmetry=kwant.TranslationalSymmetry(tz))
wirex.fill(tbhamx, shape, start=(0, 0, 0))
wirexf = wirex.finalized()

In [ ]:
type(kphamx)

In [ ]:
sympy.sympify(kphamx)

In [ ]:
x1 = wirexf.cell_hamiltonian(sparse=True)

In [ ]:
x2 = wiref.cell_hamiltonian(sparse=True, params={p: (0 if p not in ['m_0', 'E_x', 'phi_0'] else 1) for p in InAs})

In [ ]:
(-x1 - x2).data

In [ ]:
(x1 - xop).data

In [ ]:
x=0.1
np.array([[1/np.tan(x), 1/np.sin(x)], [1/np.sin(x), 1/np.tan(x)]])